LAST UPDATED ON 2023/05/02

1. Aims at evaluating the trained model

# Import Libraries

In [1]:
import os
import cv2
import copy
import random
import numpy as np

from marco import *
from network import *
from utilities import *
from visualise import *

# Define Parameters

In [126]:
#define class name to class index
CLS2IND = {'cat': 0, 
           'dog': 1}
#define class index to class name
IND2CLS = {0: 'cat', 
           1: 'dog'}

#define figure for visualization
N_COL = 4
N_ROW = 4
#number of grid in one row or column
GRID_NUM  = 7
#threshold of objectness confidence
OBJ_CONF_THRESHOLD = 0.3
#thresould of IOU for non-maximum suppression
NMS_IOU_THRESHOLD = 0.3
#thresould of IOU for evaluation
EVL_IOU_THRESHOLD = 0.5

#obtain the images in the "example_img" folder
ROOT = 'cat_and_dog_det_data'
FOLDERS = ['valid']

img_path = []
lab_path = []
for folder in FOLDERS:
    dir = os.path.join(ROOT, folder)
    data_path = os.listdir(dir)
    img_subpath = sorted([os.path.join(dir, f) for f in data_path if f.split('.')[-1] == 'jpg'])
    lab_subpath = sorted([os.path.join(dir, f) for f in data_path if f.split('.')[-1] == 'json'])
    
    img_path += img_subpath
    lab_path += lab_subpath

print("N_img: ", len(img_path))
print("N_lab: ", len(lab_path))

N_img:  367
N_lab:  367


# Initialize Detection Network

In [127]:
net = Det_Network()
net_model = net.obj_det()
net_model.load_weights('det_model/best_det_model.h5')

In [128]:
t_cat_pos = f_cat_pos = t_dog_pos = f_dog_pos = 0
dog_cnt = cat_cnt = data_cnt = 0

for img_load, lab_load in zip(img_path, lab_path):
    
    #======= load image =======#
    
    img = cv2.cvtColor(cv2.imread(img_load), cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (448, 448), interpolation=cv2.INTER_AREA)
    img = img/255.
    img_h, img_w, _ = img.shape

    draw_img = copy.deepcopy(img)
        
    #======= load label =======#
    
    lab   = json.load(open(lab_load, 'r'))
    boxes_t = np.array(lab['boxes'])
    objs_t = np.argmax(lab['class'], axis = 1)
    
    boxes_t[:, 0], boxes_t[:, 1] = boxes_t[:, 0]*img_w, boxes_t[:, 1]*img_h
    boxes_t[:, 2], boxes_t[:, 3] = boxes_t[:, 2]*img_w, boxes_t[:, 3]*img_h
    boxes_t = np.array(minmax2yolo(boxes_t, img_w = img_w, img_h = img_h))

    dog_cnt += np.sum(objs_t)
    cat_cnt += np.sum(1 - objs_t)
    
    #======= prediction =======#
    p = net_model.predict(np.expand_dims(img, 0))
    cvt_pred_mat = cvt_cell_ratio_to_img_ratio(p)

    #======= extract prediction data =======#
    box_p = []
    obj_p = []

    for i in range(GRID_NUM):
        for j in range(GRID_NUM):
            if cvt_pred_mat[0, i, j, 1] > OBJ_CONF_THRESHOLD:
                cls_ind = cvt_pred_mat[0, i, j, 0].numpy()
                tmp = [0] + list(cvt_pred_mat[0, i, j, :].numpy())
                box_p.append(tmp)
                obj_p.append(IND2CLS[cls_ind])
    box_p = np.array(box_p)
    
    #======= compute nms and record performance =======#
    if len(box_p) > 0:
        #======= compute nms =======#
        nms_boxes = np.array(compute_nms(box_p.copy(), 
                           iou_t = NMS_IOU_THRESHOLD, 
                           conf_t = OBJ_CONF_THRESHOLD))
        
        if nms_boxes.size > 0: 
            if len(nms_boxes.shape) == 1:
                nms_boxes = np.expand_dims(nms_boxes, 0)
            
            nms_objs = []
            for i in range(len(nms_boxes)):
                nms_objs.append(IND2CLS[nms_boxes[i, 1]])
            nms_objs = np.array(nms_objs)
            
         #======= record performance =======#
        s_ind = np.argsort(nms_boxes[:, 2])[::-1]
        s_nms_boxes = nms_boxes[s_ind,:]
        s_nms_objs  = nms_objs[s_ind]
        
        tbox_check = np.zeros((len(box_t)))
        pbox_check = np.zeros((len(nms_boxes),))
        
        for box_p_cnt, s_nms_box in enumerate(s_nms_boxes):
            for box_t_cnt, box_t in enumerate(boxes_t):
                #check if this ground true box is detected
                if tbox_check[box_t_cnt] == 0:
                    overlap = compute_IOU_by_union(np.expand_dims(s_nms_box[3:], 0), 
                                                   np.expand_dims(box_t, 0))
                    if overlap >= EVL_IOU_THRESHOLD and CLS2IND[s_nms_objs[box_p_cnt]] == objs_t[box_t_cnt]:
                        if objs_t[box_t_cnt] == 0:
                            t_cat_pos += 1
                        else:
                            t_dog_pos += 1

                        tbox_check[box_t_cnt] = 1
                        pbox_check[box_p_cnt] = 1
                        break
            
            #check redundant boxes
            if pbox_check[box_p_cnt] == 0:
                if CLS2IND[s_nms_objs[box_p_cnt]] == 0:
                    f_cat_pos += 1
                else:
                    f_dog_pos += 1

2023-05-02 15:05:30.529311: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 12ms/step


In [129]:
print('cat_cnt: ', cat_cnt)
print('dog_cnt: ', dog_cnt)
print('t_cat_pos: ', t_cat_pos)
print('t_dog_pos: ', t_dog_pos)
print('f_cat_pos: ', f_cat_pos)
print('f_dog_pos: ', f_dog_pos)

cat_cnt:  118
dog_cnt:  249
t_cat_pos:  104
t_dog_pos:  217
f_cat_pos:  3
f_dog_pos:  35


# Compute Recall and Precision

In [131]:
cat_p = t_cat_pos/(t_cat_pos + f_cat_pos)
cat_r = t_cat_pos/cat_cnt

dog_p = t_dog_pos/(t_dog_pos + f_dog_pos)
dog_r = t_dog_pos/dog_cnt

print(f'cat_recall: {cat_p :.3f}', )
print(f'cat_precision: {cat_r :.3f}',)
print(f'dog_recall: {dog_p :.3f}', )
print(f'dog_precision: {dog_r :.3f}', )

print(f'average precision {(cat_p + dog_p)/2 :.3f}')
print(f'average recall {(cat_r + dog_r)/2 :.3f}')

cat_recall: 0.972
cat_precision: 0.881
dog_recall: 0.861
dog_precision: 0.871
average precision 0.917
average recall 0.876
